# 爬蟲實戰 - 股票篇
學會爬蟲就可以加入網軍了嗎? 可以領1450嗎? 既然大家都被標題騙來了，就看下去吧ヾ(●゜▽゜●)

## Yahoo 股票及時報價

教學篇我們提到網站框架越新、設計越潮，爬蟲的難度就會越高。先來看看一些金融網站的設計吧~
1. Yahoo台灣 https://tw.stock.yahoo.com/
2. Yahoo金融 https://finance.yahoo.com/quote/TW/
3. 台灣證券交易所 https://www.twse.com.tw/zh/
4. Trading Economics https://tradingeconomics.com

這幾個網站對台積電(股票代號2330)的即時股價的頁面長這樣:
1. Yahoo台灣 https://tw.stock.yahoo.com/q/q?s=2330 
2. Yahoo金融 https://finance.yahoo.com/quote/2330.TW?p=2330.TW&.tsrc=fin-srch (可爬 需其他套件)
3. 台灣證券交易所 https://www.twse.com.tw/zh/page/trading/exchange/STOCK_DAY.html (為動態查詢內容 同上)
4. Trading Economics https://tradingeconomics.com/2330:tt (同上)

[](pandas.read_html)
[](https://www.twse.com.tw/exchangeReport/STOCK_DAY_AVG?response=html&date=20200220&stockNo=2330)
[](https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date=20200220&stockNo=2330&_=1582185467917)

## 目標: 擷取今日的開、高、低、收&昨收
- 今天我們的對象是台積電，目標數字所在的欄位如下圖
![](https://i.imgur.com/Rk3idyT.png)


## 開始實作
不管是多複雜的網頁爬蟲，我們一樣採取2步驟戰略:「先建立連線，再解析網頁」

首先，我們先知道台灣yahoo網站的台積電網址，才能發出http請求已建立連線。

In [2]:
import requests

url = "https://tw.stock.yahoo.com/q/q?s=2330"
header = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36"}

res = requests.get(url, header)
res

<Response [200]>

### bs4 parser > find table
第二步驟是透過BeautifulSoup套件來解析HTML網頁。

In [3]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(res.text, "html.parser")

其實解析完HTML網頁，你會發現好像沒甚麼頭緒，因為parse出來的標籤(tags)太多太亂了，還有混雜CSS、JavaScript在裡面(๑•́ ₃ •̀๑)

但是我們知道目標資料都在一個表格裡面，表格在HTML格式中代表的標籤是`<table>`，先按下F12(或是滑鼠右鍵>檢查)嘗試找出`<table>`這個標籤在哪裡吧。順帶一提，表格`<table>`中的每一列是一個`<tr>`(table row)，每列中會有許多資料`<tr>`(table data)，資料的標題是`<th>`(table header)。

```html
<table>
    <tr>
        <th>欄位名稱</th>
        <th>欄位名稱</th>
        ...
    </tr>
    <tr>
        <td>data</td>
        <td>data</td>
        ...
    </tr>
</table>
```

In [10]:
# There's 6 tables in the HTML file
len(soup.select("table"))

6

In [13]:
# Index = 2 (third one) is the one we want
table = soup.select("table")[2]
print(table)

<table border="2" width="750">
<tr bgcolor="#fff0c1">
<th align="center">股票<br/>代號</th>
<th align="center" width="55">時間</th>
<th align="center" width="55">成交</th>
<th align="center" width="55">買進</th>
<th align="center" width="55">賣出</th>
<th align="center" width="55">漲跌</th>
<th align="center" width="55">張數</th>
<th align="center" width="55">昨收</th>
<th align="center" width="55">開盤</th>
<th align="center" width="55">最高</th>
<th align="center" width="55">最低</th>
<th align="center">個股資料</th>
</tr>
<tr>
<td align="center" width="105"><a href="/q/bc?s=2330">2330台積電</a><br/><a href="/pf/pfsel?stocklist=2330;"><font size="-1">加到投資組合</font><br/></a></td>
<td align="center" bgcolor="#FFFfff" nowrap="">11:13</td>
<td align="center" bgcolor="#FFFfff" nowrap=""><b>324.5</b></td>
<td align="center" bgcolor="#FFFfff" nowrap="">324.0</td>
<td align="center" bgcolor="#FFFfff" nowrap="">324.5</td>
<td align="center" bgcolor="#FFFfff" nowrap=""><font color="#009900">▽1.0
                <td align="ce

### Try to get the latest market price

在交易時間，成交價是上一筆交易的價格，如果在09:00-13:30執行程式，會看到價格一直在變動喔! 

所以說好的爬蟲能帶來先手優勢 (ﾉ∀`*)

In [22]:
table.select("tr")[0].select("th")[2]

<th align="center" width="55">成交</th>

In [25]:
table.select("tr")[1].select("td")[2].text

'324.5'

### Get all the info we want

In [41]:
row = table.select("tr")[1].select("td")

vol  = row[6].text.replace(',', '')
open = row[8].text
high = row[9].text
low  = row[10].text
close= row[7].text
adj_close = row[2].text

In [42]:
data = {
    "open": open,
    "high": high,
    "low" : low,
    "close": close,
    "adj_close": adj_close,
    "volumn": vol
}
data

{'open': '325.5',
 'high': '326.5',
 'low': '323.0',
 'close': '325.5',
 'adj_close': '324.5',
 'volumn': '14833'}

### But...我們不只想要看台積電的行情

我們選yahoo台灣的網站來爬蟲是有原因的，觀察一下他的網址，是不是有什麼共通點呢? 若是國外Trading Economics網站，每種台股的網址就會變來變去 (我也不知道為什麼xd) 
- https://tw.stock.yahoo.com/q/q?s=2330
- https://tw.stock.yahoo.com/q/q?s=2317

為什麼要選網址相似度高、有規律性的網站呢? 一切的答案就是我們**一次想看很多標的的股價**然後還不用修改程式碼。(我就懶.jpg) 
- 2330 台積電
- 2317 鴻海
- 2884 玉山金
- more...

所以只要宣告一個list，裡面放想看的股票代碼。再寫一個函數(function)，它可以幫我們自動爬蟲list裡面的每一隻股票的行情，就完成我們的自動化爬蟲啦。
```python
# 股票列表
stocks = [2330, 2317, ...] 

# 爬蟲函式
def yahoo_stock_crawler():
    return
```

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
stocks = [2330, 2317, 2884]

# 爬蟲函式
def yahoo_stock_crawler(stock):
    return

接著我們用**迴圈**遍歷股票列表stocks，呼叫這個函式

In [ ]:
stocks = [2330, 2317, 2884]

for stock in stocks:
    yahoo_stock_crawler(stock)

### Combine with historic data

In [35]:
import pandas as pd
dfs = pd.read_html(res.text)
dfs

[    0                1
 0 NaN  資料日期: 109/02/20,                                                    0
 0  股票代號  時間  成交  買進  賣出  漲跌  張數  昨收  開盤  最高  最低  ...,             股票代號     時間     成交     買進     賣出    漲跌     張數     昨收     開盤  \
 0  2330台積電加到投資組合  14:30  325.5  325.5  326.0  ▽1.0  26923  326.5  328.0   
 
       最高     最低                個股資料  
 0  329.0  325.0  成交明細技術　新聞基本　籌碼個股健診  ,         0   1
 0  相關贊助網站 NaN
 1  相關贊助網站 NaN,         0   1
 0  相關贊助網站 NaN,                                                    0
 0  Yahoo Finance 服務條款 隱私權 台股資料來源臺灣證券交易所、臺灣期貨交易所及財...]

In [40]:
dfs[2]

,股票代號,時間,成交,買進,賣出,漲跌,張數,昨收,開盤,最高,最低,個股資料
0,2330台積電加到投資組合,14:30,325.5,325.5,326.0,▽1.0,26923,326.5,328.0,329.0,325.0,成交明細技術 新聞基本 籌碼個股健診
